# Encrypt & Upload datasets

Now that the dataset is ready, encrypt them. Because the same key has to be shared between the setup part (on a secure environment) and the deployed model (on a CVM), for semplicity use a shared key on a publicly available webserver.

In [ ]:
! curl -L https://people.redhat.com/eesposit/key.bin -o key.bin
# ! openssl rand 128 > key.bin

In [ ]:
%%bash

KEY_FILE=key.bin
DATASET_SRC=datasets
DATASET_DEST=datasets_enc

mkdir -p $DATASET_DEST

for file in $DATASET_SRC/*; do
    fname=$(basename $file)
    openssl enc -aes-256-cfb -pbkdf2 -kfile $KEY_FILE -in $file -out $DATASET_DEST/$fname.enc
done

ls $DATASET_DEST

Because the key is not needed anymore at this point, delete it. Since we are in a secure environment this is not important though.

In [ ]:
! rm -f key.bin

Install dependences to upload encrypted datasets to Azure Blob and AWS S3

In [ ]:
# Install Required Packages
!pip install azure-storage-blob boto3 > /dev/null

Upload `dataset1.csv.enc` to Azure Blob Storage

In [ ]:
# This section connects to Azure Blob Storage and uploads a local file.
# Replace placeholders with your actual Azure Storage account name and key.
from azure.storage.blob import BlobServiceClient
import os

# Azure Credentials
account_name = "mystorageacct12345"  # Replace with your storage account name
account_key = "YOUR_AZURE_ACCESS_KEY"  # Replace with your access key
container_name = "mycontainer"
blob_name = "dataset1.csv.enc"
local_file_path = "dataset1.csv.enc"
dataset_dest = "datasets_enc"

# Build the account URL
account_url = f"https://{account_name}.blob.core.windows.net"

# Connect to Azure Blob Service
blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
container_client = blob_service_client.get_container_client(container_name)

#  Upload the file to Azure
with open(dataset_dest + "/" + local_file_path, "rb") as data:
    container_client.upload_blob(name=blob_name, data=data, overwrite=True)
    print(f"Uploaded '{blob_name}' to Azure Blob Storage")

Upload `dataset2.csv.enc` to AWS S3

In [ ]:
# This section connects to AWS S3 and uploads a local file.
# Replace placeholders with your AWS access key, secret key, and bucket name.
import boto3

# AWS Credentials
aws_access_key_id = "YOUR_AWS_ACCESS_KEY"
aws_secret_access_key = "YOUR_AWS_SECRET_KEY"
aws_bucket_name = "myawsbucketaidata"
file_name = "dataset2.csv.enc"
s3_key = "dataset2.csv.enc"
dataset_dest = "datasets_enc"

# Connect to AWS S3
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Upload file to AWS S3
s3.upload_file(dataset_dest + "/" + file_name, aws_bucket_name, s3_key)
print(f"Uploaded '{file_name}' to S3 bucket '{aws_bucket_name}'")

Let's remove both datasets, we'll emulate download of encrypted datasets later in showcase phase.

In [ ]:
! rm -rf datasets_enc datasets